In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [17]:
#Import CSV from WeatherPy to create a DataFrame
df = pd.read_csv (r'../WeatherPy/output_data/cities.csv')
df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,esperance,-33.87,121.90,9.00,100,75,2.60,AU,1596485797
1,1,jamestown,42.10,-79.24,24.00,64,40,5.70,US,1596485745
2,2,aragarcas,-15.90,-52.25,31.93,21,0,2.06,BR,1596486007
3,3,rikitea,-23.12,-134.97,23.10,82,30,8.15,PF,1596485889
4,4,busselton,-33.65,115.33,8.33,84,45,5.80,AU,1596485889
...,...,...,...,...,...,...,...,...,...,...
576,576,hamilton,39.18,-84.53,28.89,54,40,1.50,US,1596485893
577,577,liepaja,56.54,21.04,17.00,93,100,0.92,LV,1596486072
578,578,konya,38.17,32.50,21.00,32,0,2.10,TR,1596486072
579,579,garissa,-0.45,39.64,24.50,60,42,9.90,KE,1596486072


# Humidity Heatmap

In [18]:
# Store latitude and longitude
cities = df[["Lat", "Lng"]]

# Store cities
humidity = df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(cities, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
#Create new DataFrame fitting weather criteria
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_df = df.loc[(df["Max Temp"] > 21) & (df["Max Temp"] < 26) & (df["Cloudiness"] == 0) & (df["Wind Speed"] < 10)
                  & (df["Humidity"] <= 50), :]
ideal_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
113,113,sarbeni,44.47,25.38,25.00,44,0,5.70,RO,1596486019
159,159,komsomolskiy,40.43,71.72,24.00,47,0,3.10,UZ,1596485937
256,256,ovalle,-30.60,-71.20,23.53,35,0,1.83,CL,1596486036
342,342,esmeraldas,-19.76,-44.31,23.00,43,0,5.70,BR,1596485732
547,547,manono,-7.30,27.42,21.92,34,0,2.25,CD,1596486068
564,564,natalio,-26.72,-55.08,25.22,48,0,0.37,PY,1596486070


In [25]:
# Store latitude and longitude in coordinates
hotel_df = ideal_df[["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"]= ""
hotel_df

C:\Users\Mabel\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,Lat,Lng,Hotel Name
113,sarbeni,RO,44.47,25.38,
159,komsomolskiy,UZ,40.43,71.72,
256,ovalle,CL,-30.60,-71.20,
342,esmeraldas,BR,-19.76,-44.31,
547,manono,CD,-7.30,27.42,
564,natalio,PY,-26.72,-55.08,


In [26]:
#Create a list to hold information
hotels = []

In [27]:
#Loop through each coordinates and save data
for row in hotel_df:
    lat = row["Lat"]
    lng = row["Lng"]
    #Geocoordinates
    target_radius = 5000
    target_type = "lodging"
    #Parameters
    params = {"radius": target_radius, "type": target_type, "key": gkey}
    params["location"] = f"{lat},{lng}"
    #Base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
#     for item in response:
#         hotel.append()
    # convert response to json
    response = response.json()
    print(response)    

TypeError: string indices must be integers

In [28]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": gkey
}
# Iterate through
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    name_address = requests.get(base_url, params=params)
    # convert to json
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

Missing field/result... skipping.


C:\Users\Mabel\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Country,Lat,Lng,Hotel Name
113,sarbeni,RO,44.47,25.38,
159,komsomolskiy,UZ,40.43,71.72,Ikathouse
256,ovalle,CL,-30.60,-71.20,Hotel Altos de Tuqui
342,esmeraldas,BR,-19.76,-44.31,Hotel Fazenda Lagoa Azul
547,manono,CD,-7.30,27.42,Hotel hewa bora
564,natalio,PY,-26.72,-55.08,Casa familia Sánchez


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [30]:
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))